<a href="https://colab.research.google.com/github/ako1983/ako-oak-llama2-finetuned-mindsdb/blob/main/lama2_ft_mindsDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch==2.5.1 torchvision==0.20.1 torchaudio==2.5.1 fastai --upgrade

In [ ]:
!pip install transformers  peft datasets accelerate --upgrade

In [ ]:
import torch
print(torch.__version__)  # Should output 2.5.1


2.5.1+cu124


In [ ]:
!nvidia-smi  # Check if A100 is active


Mon Mar 10 03:33:18 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   34C    P0             43W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
DATA_PATH = "/content/drive/MyDrive/synthetic_qa.jsonl"
OUTPUT_DIR = "/content/drive/MyDrive/llama2-finetune/"


In [ ]:
from google.colab import files
uploaded = files.upload()  # Select synthetic_qa.jsonl

Saving synthetic_qa.jsonl to synthetic_qa.jsonl


In [ ]:
from google.colab import files
uploaded = files.upload()  #q_lora

Saving colab_train.py to colab_train.py


In [ ]:
from huggingface_hub import login

login()

In [ ]:
def tokenize_function(examples):
    tokenized_inputs = tokenizer(examples["text"], padding="max_length", max_length=512, truncation=True)

    # Shift input tokens to create labels (this is how LLMs learn)
    tokenized_inputs["labels"] = tokenized_inputs["input_ids"].copy()

    return tokenized_inputs


In [ ]:
!python colab_train.py

2025-03-10 03:42:35.655588: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-10 03:42:35.674005: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741578155.695977    5872 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741578155.702693    5872 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-10 03:42:35.725473: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_path = "/content/llama2-finetune"
model = AutoModelForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

def chat(prompt):
    inputs = tokenizer(prompt, return_tensors="pt")
    output = model.generate(**inputs, max_length=200, temperature=0.7, top_p=0.9)
    return tokenizer.decode(output[0], skip_special_tokens=True)

print(chat("What is the purpose of handlers in MindsDB?"))


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

What is the purpose of handlers in MindsDB?
Answer: Handlers in MindsDB allow users to connect their data sources to MindsDB and interact with them directly using SQL commands.



In [ ]:
from huggingface_hub import notebook_login

notebook_login()




In [ ]:
model.push_to_hub("ako-oak/llama2-finetuned-mindsdb")
tokenizer.push_to_hub("ako-oak/llama2-finetuned-mindsdb")


README.md:   0%|          | 0.00/31.0 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/16.8M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ako-oak/llama2-finetuned-mindsdb/commit/a8d983f813beca3c08338ae9c53319fc7f5d6528', commit_message='Upload tokenizer', commit_description='', oid='a8d983f813beca3c08338ae9c53319fc7f5d6528', pr_url=None, repo_url=RepoUrl('https://huggingface.co/ako-oak/llama2-finetuned-mindsdb', endpoint='https://huggingface.co', repo_type='model', repo_id='ako-oak/llama2-finetuned-mindsdb'), pr_revision=None, pr_num=None)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Copy model to Google Drive
!cp -r /content/llama2-finetune /content/drive/MyDrive/llama2-finetune


Mounted at /content/drive
